## Module 1 Homework

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

### Q1. Downloading the data

We'll use "Yellow Taxi Trip Records". Download the data for January and February 2023. Read the data for January. How many columns are there?

In [2]:
raw_train_df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [3]:
raw_train_df.shape

(3066766, 19)

### Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

In [4]:
raw_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
raw_train_df["duration"] = (raw_train_df["tpep_dropoff_datetime"] - raw_train_df["tpep_pickup_datetime"]).dt.total_seconds() / 60

In [6]:
raw_train_df["duration"].describe()["std"]

42.59435124195458

### Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers.

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [7]:
# train_df = raw_train_df[(raw_train_df["duration"] >= 1) & (raw_train_df["duration"] <= 60)].copy()
train_df = raw_train_df[raw_train_df["duration"].between(1, 60)].copy()

In [8]:
(1 - ((len(raw_train_df) - len(train_df)) / len(raw_train_df))) * 100

98.1220282212598

In [9]:
# tmp_train_df = raw_train_df[(raw_train_df["duration"] >= 1) & (raw_train_df["duration"] <= 60)]

### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [10]:
# Convert int IDs into str IDs
categorical_features = ["PULocationID", "DOLocationID"]
train_df[categorical_features] = train_df[categorical_features].astype(str)

In [11]:
train_data = train_df[categorical_features].to_dict(orient="records")

v = DictVectorizer()
X_train = v.fit_transform(train_data)

In [12]:
X_train.shape

(3009173, 515)

### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

* Train a vanilla linear regression model with default parameters
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [13]:
target = "duration"
y_train = train_df[target].values

In [14]:
y_train.shape

(3009173,)

In [15]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [16]:
y_pred = reg.predict(X_train)

rmse = root_mean_squared_error(y_train, y_pred)
print(f"RMSE on train: {rmse}")

RMSE on train: 7.649261932106969


### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [17]:
raw_validation_df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [18]:
raw_validation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2913955 entries, 0 to 2913954
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [19]:
# Data preprocessing
validation_df = raw_validation_df.copy()

validation_df["duration"] = (validation_df["tpep_dropoff_datetime"] - validation_df["tpep_pickup_datetime"]).dt.total_seconds() / 60
validation_df = validation_df[validation_df["duration"].between(1, 60)].copy()

In [20]:
# Feature Engineering
validation_df[categorical_features] = validation_df[categorical_features].astype(str)
validation_data = validation_df[categorical_features].to_dict(orient="records")

X_validation = v.transform(validation_data)

In [21]:
X_validation.shape

(2855951, 515)

In [22]:
y_validation = validation_df[target].values
y_validation.shape

(2855951,)

In [24]:
# Model evaluation
y_pred = reg.predict(X_validation)

rmse = root_mean_squared_error(y_validation, y_pred)
print(f"RMSE on train: {rmse}")

RMSE on train: 7.811818743246608
